In [1]:
# Objective for this Dataset:- we will use various predictive models to see how accurate, 
# they are in detecting whether we can correctly predict which comapnies will face bankrptcy or not.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action="ignore")
plt.rcParams['figure.figsize']=[15,7]

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc, roc_curve, accuracy_score, recall_score, 
                             classification_report, f1_score, average_precision_score, precision_recall_fscore_support, 
                             roc_auc_score)
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier 
import xgboost as XGB
import lightgbm as lgb

In [3]:
import os
os.getcwd()

'C:\\Users\\VICTORS\\Desktop\\Project 2'

In [4]:
data = pd.read_csv("_BANKRUPTCY_PREDICTION.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '_BANKRUPTCY_PREDICTION.csv'

In [ ]:
# Computing the Descriptive statistics of our numerical features
data.describe()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
# Checking Nan presence

[print(col) for col in data if data[col].isna().sum() > 0]

In [ ]:
# find the uniqueness in the dataset
for i in data.columns:
    print("The no.of unique values in",i,"are: ",data.loc[:,i].nunique())

In [ ]:
# check for duplicates
data.duplicated().sum()

In [ ]:
# Considering the project task, and now that we have a general overview of our data, we need 
# focus our attention on the labels: which are the financially stable and unstable companies? 
# Let's take a look:

print(data['Bankrupt'].value_counts())
print('-'* 30)
print('Financially stable: ', round(data['Bankrupt'].value_counts()[0]/len(data) * 100,2), '% of the dataset')
print('Financially unstable: ', round(data['Bankrupt'].value_counts()[1]/len(data) * 100,2), '% of the dataset')

In [ ]:
# Checking labels distributions
data['Bankrupt'].value_counts().plot.pie(autopct="%2f")

# we can clearly see how our labels are strongly unbalanced, and this is a the main obstacle that we need to solve to obtain good performances.

In [ ]:
# EDA & Visualisation

for i in data.columns:
    plt.figure(figsize=(15,3))
    sns.distplot(data.loc[:,i],hist=False)
    plt.grid()
plt.show()

In [ ]:
# Let Find out if our data is skewed or not.
for i in data.columns:
    print("The skewness of",i,"is: ",data.loc[:,i].skew())
    print("------------------------------------------------------------------------------------------------")

In [ ]:
skew=[]
for i in data.columns:
    skew.append(data.loc[:,i].skew())

In [ ]:
skew_dict={'features': data.columns, 'skewness':skew}
skew_data= pd.DataFrame(skew_dict)
skew_data

In [ ]:
sns.barplot(x=skew_data['features'], y=skew_data['skewness'])
plt.axhline(y=3, color='r')
plt.axhline(y=-3, color='r')
plt.axhline(y=0, color='k')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Now a Glance on histogram and Heatmap for the correlation analysis..
data.hist(figsize = (35,30), bins = 50)
plt.show()

In [ ]:
# Correlation Heatmap (Spearman)
f, ax = plt.subplots(figsize=(30, 25))
mat = data.corr('spearman')
mask = np.triu(np.ones_like(mat, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(mat, mask=mask, cmap=cmap, vmax=1, center=0,# annot = True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

In [ ]:
# For More Clear insights We can use Boxplot for numerical features.
plt.figure(figsize = (20,20))
ax =sns.boxplot(data = data, orient="h")
ax.set_title('Description of ranges of data', fontsize = 18)
ax.set(xscale="log")
plt.show()

In [ ]:
# Now lets plot those varibale whose its own value in any company's growth or demotion..

f, axes = plt.subplots(ncols=4, figsize=(24,6))

sns.boxplot(x='Bankrupt', y=" Net Income to Total Assets", data=data, ax=axes[0])
axes[0].set_title('Bankrupt vs Net Income to Total Assets')

sns.boxplot(x='Bankrupt', y=" Total debt/Total net worth", data=data, ax=axes[1]) 
axes[1].set_title('Bankrupt vs Tot Debt/Net worth Correlation')

sns.boxplot(x='Bankrupt', y=" Debt ratio %", data=data, ax=axes[2])
axes[2].set_title('Bankrupt vs Debt ratio Correlation')

sns.boxplot(x='Bankrupt', y=" Net worth/Assets", data=data, ax=axes[3])  
axes[3].set_title('Bankrupt vs Net Worth/Assets Correlation') 

plt.show()

In [ ]:
# Because Index 4 out of bounds error appear i.e why we are plotting in two times.
f, axes = plt.subplots(ncols=4, figsize=(24,6))

sns.boxplot(x='Bankrupt', y=" Working Capital to Total Assets", data=data, ax=axes[0])
axes[0].set_title('Bankrupt vs  working capital to total assets')

sns.boxplot(x='Bankrupt', y=" Cash/Total Assets", data=data, ax=axes[1])
axes[1].set_title('Bankrupt vs cash / total assets')

sns.boxplot(x='Bankrupt', y=" Current Liability to Assets", data=data, ax=axes[2])
axes[2].set_title('Bankrupt vs current liability to assets')

sns.boxplot(x='Bankrupt', y=" Retained Earnings to Total Assets", data=data, ax=axes[3])
axes[3].set_title('Bankrupt vs  Retained Earnings to Total Assets')

plt.show()

In [ ]:
# Plotting the feature distributions for close to bankrputcy companies

f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(24, 6))

cash_flow_rate = data[' Net Income to Total Assets'].loc[data['Bankrupt'] == 1].values
sns.distplot(cash_flow_rate,ax=ax1, color='#FB8861')
ax1.set_title(' Net Income to Total Assets \n (Unstable companies)', fontsize=14)

tot_debt_net = data[' Total debt/Total net worth'].loc[data['Bankrupt'] == 1].values
sns.distplot(tot_debt_net ,ax=ax2, color='#56F9BB')
ax2.set_title('total debt/tot net worth \n (Unstable companies)', fontsize=14)


debt_ratio = data[' Debt ratio %'].loc[data['Bankrupt'] == 1].values
sns.distplot(debt_ratio,ax=ax3, color='#C5B3F9')
ax3.set_title('debt_ratio \n (Unstable companies)', fontsize=14)

net_worth_assets = data[' Net worth/Assets'].loc[data['Bankrupt'] == 1].values
sns.distplot(net_worth_assets,ax=ax4, color='#C5B3F9')
ax4.set_title('net worth/assets \n (Unstable companies)', fontsize=14)

plt.show()

In [ ]:
f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(24, 6))

working_cap = data[' Working Capital to Total Assets'].loc[data['Bankrupt'] == 1].values
sns.distplot(working_cap,ax=ax1, color='#FB8861')
ax1.set_title('working capitals to total assets \n (Unstable companies)', fontsize=14)

cash_tot_assets = data[' Cash/Total Assets'].loc[data['Bankrupt'] == 1].values
sns.distplot(cash_tot_assets ,ax=ax2, color='#56F9BB')
ax2.set_title('cash/total assets \n (Unstable companies)', fontsize=14)


asset_liab = data[' Current Liability to Assets'].loc[data['Bankrupt'] == 1].values
sns.distplot(asset_liab,ax=ax3, color='#C5B3F9')
ax3.set_title('liability to assets \n (Unstable companies)', fontsize=14)

operating_funds = data[' Retained Earnings to Total Assets'].loc[data['Bankrupt'] == 1].values
sns.distplot(operating_funds,ax=ax4, color='#C5B3F9')
ax4.set_title('retain earnings to total assets \n (Unstable companies)', fontsize=14)

In [ ]:
# will try to remove the most extreme outliers (note that you can also impute them with the mean or
# the median instead of removing them). This should increase our models' performance.

#outlier removal...
def outliers_removal(feature,feature_name,dataset):
    
    # Identify 25th & 75th quartiles

    q25, q75 = np.percentile(feature, 25), np.percentile(feature, 75)
    print('Quartile 25: {} | Quartile 75: {}'.format(q25, q75))
    feat_iqr = q75 - q25
    print('iqr: {}'.format(feat_iqr))
    
    feat_cut_off = feat_iqr * 1.5
    feat_lower, feat_upper = q25 - feat_cut_off, q75 + feat_cut_off
    print('Cut Off: {}'.format(feat_cut_off))
    print(feature_name +' Lower: {}'.format(feat_lower))
    print(feature_name +' Upper: {}'.format(feat_upper))
    
    outliers = [x for x in feature if x < feat_lower or x > feat_upper]
    print(feature_name + ' outliers for close to bankruptcy cases: {}'.format(len(outliers)))
    #print(feature_name + ' outliers:{}'.format(outliers))

    dataset = dataset.drop(dataset[(dataset[feature_name] > feat_upper) | (dataset[feature_name] < feat_lower)].index)
    print('-' * 65)
    
    return dataset

for col in data:
    new_df = outliers_removal(data[col],str(col),data)

In [ ]:
f,(ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24,6))

# Boxplots with outliers removed

sns.boxplot(x="Bankrupt", y=" Net Income to Total Assets", data=new_df,ax=ax1) 
ax1.set_title("Net Income to Total Assets \n Reduction of outliers", fontsize=14)

sns.boxplot(x="Bankrupt", y=" Total debt/Total net worth", data=new_df,ax=ax2) 
ax2.set_title("total debt/total net worth \n Reduction of outliers", fontsize=14)

sns.boxplot(x="Bankrupt", y=" Debt ratio %", data=new_df,ax=ax3) 
ax3.set_title("debt ratio % \n Reduction of outliers", fontsize=14)

sns.boxplot(x="Bankrupt", y=' Net worth/Assets', data=new_df,ax=ax4) 
ax4.set_title("net worth/assets \n Reduction of outliers", fontsize=14)
        
plt.show()

In [ ]:
f,(ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24,6))

# Boxplots with outliers removed

sns.boxplot(x="Bankrupt", y=' Working Capital to Total Assets', data=new_df,ax=ax1) 
ax1.set_title("working capital to total assets \n Reduction of outliers", fontsize=14)

sns.boxplot(x="Bankrupt", y=' Cash/Total Assets', data=new_df,ax=ax2) 
ax2.set_title("cash / total assets \n Reduction of outliers", fontsize=14)

sns.boxplot(x="Bankrupt", y=' Current Liability to Assets', data=new_df,ax=ax3) 
ax3.set_title("current liability to assets \n Reduction of outliers", fontsize=14)

sns.boxplot(x="Bankrupt", y=' Retained Earnings to Total Assets', data=new_df,ax=ax4) 
ax4.set_title("Retained Earnings to Total Assets \n Reduction of outliers", fontsize=14)
plt.show()

In [ ]:
def log_trans(d):
    
    for col in d:
        skew = d[col].skew()
        if skew > 0.5 or skew < -0.5:
            d[col] = np.log1p(d[col])
        else:
            continue
            
    return d

data_norm = log_trans(new_df)

In [ ]:
# Plotting Boxplots of the preprocessed numerical features

plt.figure(figsize = (20,20))
ax =sns.boxplot(data = data_norm, orient="h")
ax.set_title('Bank Data Preprocessed Boxplots', fontsize = 18)
ax.set(xscale="log")
plt.show()

In [ ]:
def preprocess(df):
    df = df.copy()
    y = df['Bankrupt']
    X = df.drop('Bankrupt', axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state=2)
    
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess(data)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y_test.value_counts() / len(y_test)

In [ ]:
var_models = {
    "Logistic Regression": LogisticRegression(solver = "liblinear", l1_ratio = 0.4),
    "K-Nearest Neighbors": KNeighborsClassifier(weights='distance', metric='euclidean'),
    "Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (LinearSVC)": LinearSVC(C = 0.3),
    "Support Vector Machine (kernel SVM)": SVC(kernel = 'rbf', random_state = 0),
    "MLPClassifier": MLPClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(loss = "exponential"),
    "AdaBoost Classifier": AdaBoostClassifier(n_estimators = 60, learning_rate = 0.2),
    "XgBoost Classifier": XGB.XGBClassifier(),
    "CatBoost Classifier": CatBoostClassifier(),
    "LightGBM": LGBMClassifier()
}

for name, model in var_models.items():
  model.fit(X_train, y_train)
  print(name + ' was completed.')

In [ ]:
var_results = []

for name, model in var_models.items():
  results = model.score(X_test, y_test)
  var_results.append(results)

  print('----------     '+name+'     ----------')

  model = var_models[name]
  y_test_pred = model.predict(X_test)
  print(metrics.classification_report(y_test_pred, y_test))
  print(name + ': {:.4f}%'.format(results * 100))
  print('Accuracy :{0:0.4f}'.format(metrics.accuracy_score(y_test_pred , y_test))) 
  print('AUC : {0:0.4f}'.format(metrics.roc_auc_score(y_test_pred , y_test)))
  print('Precision : {0:0.4f}'.format(metrics.precision_score(y_test_pred , y_test)))
  print('Recall : {0:0.4f}'.format(metrics.recall_score(y_test_pred , y_test)))
  print('F1 : {0:0.4f}'.format(metrics.f1_score(y_test_pred , y_test)))

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)

for i in var_models:
    y_pred = var_models[i].predict(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
    roc_auc = auc(fpr, tpr)
    precision, recall, th = precision_recall_curve(y_test, y_pred,pos_label=1)
    plt.plot(fpr, tpr, label= i + ' : {:.4f}'.format(roc_auc_score(y_test, y_pred)))

plt.plot([0,1], [0,1], 'k--' )
plt.axis([-0.01, 1, 0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve for Predicting a Bankruptcy')
plt.legend(loc=4)
plt.show()

In [ ]:
a = 1

fig = plt.figure(figsize=(25,15))

for name, model in var_models.items():
    results = model.score(X_test, y_test)
    model = var_models[name]
    y_test_pred = model.predict(X_test)
    arg_test = {'y_true':y_test, 'y_pred':y_test_pred}

    conf_mx = confusion_matrix(y_test, y_test_pred)

    heat_cm = pd.DataFrame(conf_mx, columns=np.unique(y_test), index = np.unique(y_test))
    heat_cm.index.name = 'Actual'
    heat_cm.columns.name = 'Predicted'

    plt.subplot(4, 3, a)
    fig.subplots_adjust(wspace=0.45, hspace= 0.45)
    sns.heatmap(heat_cm, annot=True, fmt='.2f', square=True, annot_kws={"size": 14}, cmap = 'PuRd').set_title(name, fontsize = 20)
    a = a + 1

plt.show()

In [ ]:
#################################################################################
# Till now it we are working with original data with any manupulation..
# Now with Smote
sm = SMOTE(random_state = 42)
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)

In [ ]:
sns.countplot(x = y_train_smote)
plt.title('Bankrupt')

In [ ]:
var_models_smote = {
    "Logistic Regression": LogisticRegression(solver = "liblinear", l1_ratio = 0.4),
    "K-Nearest Neighbors": KNeighborsClassifier(weights='distance', metric='euclidean'),
    "Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (LinearSVC)": LinearSVC(C = 0.3),
    "Support Vector Machine (SVC)": SVC(),
    "MLPClassifier": MLPClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(loss = "exponential"),
    "AdaBoost Classifier": AdaBoostClassifier(n_estimators = 60, learning_rate = 0.2),
    "XgBoost Classifier": XGB.XGBClassifier(),
    "CatBoost Classifier": CatBoostClassifier(),
    "LightGBM": LGBMClassifier()
}

for name, model in var_models_smote.items():
  model.fit(X_train_smote, y_train_smote)
  print(name + ' was completed.')

In [ ]:
var_results_smote = []

for name, model in var_models_smote.items():
  results_smote = model.score(X_test, y_test)
  var_results_smote.append(results_smote)

  print('----------     '+name+'     ----------')

  model = var_models_smote[name]
  y_test_pred_smote = model.predict(X_test)
  print(metrics.classification_report(y_test , y_test_pred_smote))
  print('Accuracy : {:0.4f}'.format(metrics.accuracy_score(y_test , y_test_pred_smote))) 
  print('AUC : {0:0.4f}'.format(metrics.roc_auc_score(y_test , y_test_pred_smote)))
  print('Precision : {0:0.4f}'.format(metrics.precision_score(y_test , y_test_pred_smote)))
  print('Recall : {0:0.4f}'.format(metrics.recall_score(y_test , y_test_pred_smote)))
  print('F1 : {0:0.4f}'.format(metrics.f1_score(y_test , y_test_pred_smote)))

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)

for i in var_models_smote:
    y_pred = var_models_smote[i].predict(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
    roc_auc = auc(fpr, tpr)
    precision, recall, th = precision_recall_curve(y_test, y_pred,pos_label=1)
    plt.plot(fpr, tpr, label= i + ' : {:.4f}'.format(roc_auc_score(y_test, y_pred)))

plt.plot([0,1], [0,1], 'k--' )
plt.axis([-0.01, 1, 0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve for Predicting a Bankruptcy')
plt.legend(loc=4)
plt.show()

In [ ]:
a = 1

fig = plt.figure(figsize=(25,15))

for name, model in var_models_smote.items():
    results = model.score(X_test, y_test)
    model = var_models_smote[name]
    y_test_pred = model.predict(X_test)
    arg_test = {'y_true':y_test, 'y_pred':y_test_pred}

    conf_mx = confusion_matrix(y_test, y_test_pred)

    heat_cm = pd.DataFrame(conf_mx, columns=np.unique(y_test), index = np.unique(y_test))
    heat_cm.index.name = 'Actual'
    heat_cm.columns.name = 'Predicted'

    plt.subplot(4, 3, a)
    fig.subplots_adjust(wspace=0.45, hspace= 0.45)
    sns.heatmap(heat_cm, annot=True, fmt='.2f', square=True, annot_kws={"size": 14}, cmap = 'PuRd').set_title(name, fontsize = 20)
    a = a + 1

plt.show()

In [ ]:
#mlp = MLPClassifier()

#params = [{'activation':['tanh','identity','logistic','relu'],'alpha': np.logspace(0.001,0.01,num=100),'solver': ['lbfgs','sgd','adam']}]
          
#grd = GridSearchCV(estimator=mlp,param_grid=params,cv=3)

#grd_model = grd.fit(xtrain,ytrain)
          
#grd_model.best_params_

In [ ]:
#  The result of GridSearchCV is giving the following results:
# 'activation': 'tanh', 'alpha': 1.0058783356953453, 'solver': 'sgd'

In [ ]:
# At this point, we can say lightbgm and RandomForest is giving good result compare to others..
# so we will look into feature importance in this model.
rf = RandomForestClassifier()
rf_model= rf.fit(X_train_smote, y_train_smote)
rf_model.feature_importances_

In [ ]:
importances = pd.DataFrame({'Features':data.drop('Bankrupt', axis=1).columns, 'Feature Importances': rf_model.feature_importances_})
importances

In [ ]:
imp = importances.sort_values(by='Feature Importances',ascending=False)
imp

In [ ]:
plt.figure(figsize=(18,10))
sns.barplot(y='Feature Importances',x='Features',data=imp)
plt.xticks(rotation=90)
plt.show()

In [ ]:
imp['Features'][0:9].values

In [ ]:
# The above cell's varibale are some of the most important factors which result in the bankruptcy of a company/organisation.